In [ ]:
#Worklife Adjustment Calculator
#Use this cell to calculate the worklife factor, this is the WLE that is divided by the YFS.

# Function to calculate worklife factor as a percentage
def calculate_worklife_factor():
    try:
        # Ask for Worklife Expectancy (WLE) years
        wle_years = float(input("Enter the Worklife Expectancy (WLE) years (can be a decimal): "))

        # Ask for Years to Final Separation (YFS) years
        yfs_years = float(input("Enter the Years to Final Separation (YFS) years (can be a decimal): "))

        # Validate input
        if yfs_years == 0:
            print("YFS years cannot be zero. Please enter a valid number.")
            return

        # Calculate the worklife factor as a percentage
        worklife_factor = (wle_years / yfs_years) * 100

        # Print the result
        print(f"The Worklife Factor is: {worklife_factor:.2f}%")

    except ValueError:
        print("Invalid input. Please enter numeric values for WLE and YFS years.")

# Run the function
if __name__ == "__main__":
    calculate_worklife_factor()


In [ ]:
#AEF CALCULATOR
#Use this cell once all of the factors have been determined, this will provide the total AEF factor needed for the tables

import pandas as pd
import sys
from IPython.display import display

def get_percentage_input(prompt, default=None):
    while True:
        try:
            user_input = input(prompt)
            if user_input.strip() == "" and default is not None:
                return default
            value = float(user_input)
            if 0 <= value <= 100:
                return value
            else:
                print("Please enter a percentage between 0 and 100.")
        except ValueError:
            print("Invalid input. Please enter a numeric value.")

# Initialize variables with default values
gross_earnings_base = 100.0
personal_type = "None"

# Step-by-step user input
gross_earnings_base = get_percentage_input("Enter the Gross Earnings Base (default 100%): ", default=100.0)
worklife_adjustment = get_percentage_input("Enter the Worklife Adjustment (%): ")
unemployment_factor = get_percentage_input("Enter the Unemployment Factor (%): ")
fringe_benefit = get_percentage_input("Enter the Fringe Benefit percentage (%): ")
tax_liability = get_percentage_input("Enter the Tax Liability (%): ")

wrongful_death = input("Is this a wrongful death matter? (yes/no): ").strip().lower()
if wrongful_death == "yes":
    personal_type = input("Is it 'personal maintenance' or 'personal consumption'?: ").strip().lower()
    personal_percentage = get_percentage_input(f"Enter the percentage for {personal_type} (%): ")
else:
    personal_percentage = 0.0

# Perform calculations based on formula
GE = gross_earnings_base
WLE = worklife_adjustment / 100
UF = unemployment_factor / 100
FB = fringe_benefit / 100
TL = tax_liability / 100
PC = personal_percentage / 100

# Applying the formula: AIF = {[((GE x WLE) (1 - UF)) (1 + FB)] - [(GE x WLE) (1 - UF)] (TL)} (1 - PC)
base_adjustment = GE * WLE * (1 - UF)
fringe_adjusted = base_adjustment * (1 + FB)
tax_adjustment = base_adjustment * TL
final_adjusted = (fringe_adjusted - tax_adjustment) * (1 - PC)

total_factor = round(final_adjusted / GE * 100, 2)

# Prepare data for display
data = {
    "Step": [
        "Gross Earnings Base",
        "x WorkLife Adjustment",
        "x (1 - Unemployment Factor)",
        "= Adjusted Base Earnings",
        "x (1 - Tax Liability)",
        "x (1 + Fringe Benefit)",
        "x (1 - Personal Maintenance/Consumption)",
        "= Fringe Benefits/Tax Adjusted Earnings Base",
        "AEF (Adjusted Earnings Factor)",
    ],
    "Percentage": [
        "100.00%",
        f"{worklife_adjustment:.2f}%",
        f"{100 - unemployment_factor:.2f}%",
        f"{base_adjustment:.2f}%",
        f"{100 - tax_liability:.2f}%",
        f"{100 + fringe_benefit:.2f}%",
        f"{100 - personal_percentage:.2f}% ({personal_type.capitalize() if personal_type != 'None' else 'N/A'})",
        f"{fringe_adjusted:.2f}%",
        f"{total_factor:.2f}%",
    ]
}

# Display results in a table format
df = pd.DataFrame(data)
print("\nAdjusted Earnings Factor Calculation:")
print(df.to_string(index=False))

# Save results to a CSV file
csv_save_path = "adjusted_earnings_factor_calculation.csv"
df.to_csv(csv_save_path, index=False)
print(f"\nThe calculation has been saved to '{csv_save_path}'. You can open the file and copy the table as needed.")


In [5]:
#EARNINGS CALCULATOR

import pandas as pd
import os

def compute_earnings_table(
    start_date,
    end_date,
    wage_base,
    residual_base,
    growth_rate,
    discount_rate,
    adjustment_factor,
    date_of_birth=None,
    reference_start=None
):
    """
    Computes a year-by-year earnings table (with partial years) by subtracting any residual
    earning capacity from the projected wage base, yielding a "net wage base."

    Returns:
      - df: DataFrame with columns:
          [Year, Age, Portion of Year (%),
           Wage Base (Net), Gross Earnings, Adjusted Earnings, Present Value]
      - final_main_wage_base: the main wage base after annual growth through the final year
      - final_residual_base: the residual base after annual growth through the final year

    Parameters:
      start_date (str): e.g. "YYYY-MM-DD" (beginning of this period)
      end_date (str): e.g. "YYYY-MM-DD" (end of this period)
      wage_base (float): Starting projected wage base for this period
      residual_base (float): Starting residual earning capacity for this period
      growth_rate (float): e.g. 0.035 for 3.5% annual growth
      discount_rate (float): e.g. 0.05 for 5% annual discount
      adjustment_factor (float): e.g. 0.8854 (88.54%)
      date_of_birth (str or None): For age calculation, format "YYYY-MM-DD"
      reference_start (pd.Timestamp or None): date from which discounting begins;
          if None, defaults to start_date
    """

    # Convert input strings to Timestamps
    start_dt = pd.to_datetime(start_date)
    end_dt = pd.to_datetime(end_date)
    dob = pd.to_datetime(date_of_birth) if date_of_birth else None

    # If no reference_start is given, default to the period's start date
    if reference_start is None:
        reference_start = start_dt

    # Prepare data dictionary
    data = {
        "Year": [],
        "Age": [],
        "Portion of Year (%)": [],
        "Wage Base": [],          # Net (Main - Residual)
        "Gross Earnings": [],
        "Adjusted Earnings": [],
        "Present Value": []
    }

    current_wage_base = wage_base
    current_residual_base = residual_base
    current_year = start_dt.year

    final_main_wage_base = wage_base
    final_residual_base = residual_base

    while True:
        # Build full calendar-year window
        year_start = pd.Timestamp(year=current_year, month=1, day=1)
        year_end = pd.Timestamp(year=current_year, month=12, day=31)

        # Align first segment's start
        if year_start < start_dt:
            year_start = start_dt
        # Align last segment's end
        if year_end > end_dt:
            year_end = end_dt

        # Stop if invalid segment
        if year_start > year_end:
            break

        # Portion of year calculation
        days_in_period = (year_end - year_start).days + 1  # inclusive
        portion_of_year = days_in_period / 365.0

        # Calculate Age (integer) at the beginning of this segment
        if dob is not None:
            age_this_year = (year_start - dob).days // 365
        else:
            age_this_year = ""

        # Net wage base = (main - residual), clamped to zero
        net_wage_base = max(current_wage_base - current_residual_base, 0)

        # Gross Earnings for partial year
        gross_earnings = net_wage_base * portion_of_year

        # Adjusted Earnings
        adjusted_earnings = gross_earnings * adjustment_factor

        # Present Value discounting from reference_start
        years_from_ref = (year_start - reference_start).days / 365.0
        present_value = adjusted_earnings / ((1 + discount_rate) ** years_from_ref)

        # Append to data
        data["Year"].append(current_year)
        data["Age"].append(age_this_year)
        data["Portion of Year (%)"].append(f"{portion_of_year * 100:.2f}%")
        data["Wage Base"].append(f"${net_wage_base:,.2f}")
        data["Gross Earnings"].append(f"${gross_earnings:,.2f}")
        data["Adjusted Earnings"].append(f"${adjusted_earnings:,.2f}")
        data["Present Value"].append(f"${present_value:,.2f}")

        # Grow both main and residual base for next year
        current_wage_base *= (1 + growth_rate)
        current_residual_base *= (1 + growth_rate)
        current_year += 1

        # If we've reached end_dt, record final bases and break
        if year_end == end_dt:
            final_main_wage_base = current_wage_base
            final_residual_base = current_residual_base
            break

    # Construct the DataFrame
    df = pd.DataFrame(data)
    return df, final_main_wage_base, final_residual_base


# ===========================================
#              MAIN SCRIPT
# ===========================================
print("Welcome to the Post-Trial Earnings Calculator (With Residual Offset)!")

# 1) Gather user inputs
first_name = input("Enter the first name: ")
last_name = input("Enter the last name: ")
date_of_birth = input("Enter the date of birth (YYYY-MM-DD): ")
date_of_injury = input("Enter the date of injury (YYYY-MM-DD): ")
date_of_report = input("Enter the date of report (YYYY-MM-DD): ")

growth_rate = float(input("Enter the future growth rate (as a percentage, e.g., 3.5 for 3.5%): ")) / 100
discount_rate = float(input("Enter the discount rate (as a percentage, e.g., 5 for 5%): ")) / 100
adjustment_factor = float(input("Enter the adjusted earnings factor (e.g., 88.54 for 88.54%): ")) / 100
starting_wage_base = float(input("Enter the starting wage base (e.g., 46748): "))

residual_answer = input("Is there residual earning capacity? (y/n): ").strip().lower()
if residual_answer == "y":
    starting_residual_base = float(input("Enter the starting residual earning capacity (e.g., 20000): "))
else:
    starting_residual_base = 0.0

wle_years = float(input("Enter the Work Life Expectancy (WLE) in years (e.g., 47.3): "))

# 2) Derive the "retirement" date from date_of_report + WLE
report_timestamp = pd.to_datetime(date_of_report)
delta_days = round(wle_years * 365.25)
estimated_retirement_dt = report_timestamp + pd.Timedelta(days=delta_days)

# 3) Pre-Injury Table (date_of_injury -> date_of_report)
pre_injury_df, final_wage_pre_injury, final_resid_pre_injury = compute_earnings_table(
    start_date=date_of_injury,
    end_date=date_of_report,
    wage_base=starting_wage_base,
    residual_base=starting_residual_base,
    growth_rate=growth_rate,
    discount_rate=discount_rate,
    adjustment_factor=adjustment_factor,
    date_of_birth=date_of_birth,
    reference_start=pd.to_datetime(date_of_injury)  # discount from injury date
)

# 4) Post-Injury Table (date_of_report -> retirement)
post_injury_df, final_wage_post_injury, final_resid_post_injury = compute_earnings_table(
    start_date=date_of_report,
    end_date=estimated_retirement_dt,
    wage_base=final_wage_pre_injury,
    residual_base=final_resid_pre_injury,
    growth_rate=growth_rate,
    discount_rate=discount_rate,
    adjustment_factor=adjustment_factor,
    date_of_birth=date_of_birth,
    reference_start=pd.to_datetime(date_of_report)  # discount from report date
)

# 5) Summaries
def sum_currency_column(values):
    return sum(float(v.replace("$", "").replace(",", "")) for v in values)

pre_injury_adj_total = sum_currency_column(pre_injury_df["Adjusted Earnings"])
pre_injury_pv_total  = sum_currency_column(pre_injury_df["Present Value"])

post_injury_adj_total = sum_currency_column(post_injury_df["Adjusted Earnings"])
post_injury_pv_total  = sum_currency_column(post_injury_df["Present Value"])

# 6) Export to Excel
file_name = f"{first_name}_{last_name}_Earnings_Tables.xlsx"
file_path = os.path.join(os.getcwd(), file_name)

with pd.ExcelWriter(file_path) as writer:
    # Pre-Injury
    pre_injury_df.to_excel(writer, index=False, sheet_name="Pre-Injury")

    summary_pre = pd.DataFrame({
        "Year": ["Total"],
        "Age": [""],
        "Portion of Year (%)": [""],
        "Wage Base": [""],
        "Gross Earnings": [""],
        "Adjusted Earnings": [f"${pre_injury_adj_total:,.2f}"],
        "Present Value": [f"${pre_injury_pv_total:,.2f}"]
    })
    summary_pre.to_excel(writer, index=False, header=False, sheet_name="Pre-Injury",
                         startrow=len(pre_injury_df) + 2)

    # Post-Injury
    post_injury_df.to_excel(writer, index=False, sheet_name="Post-Injury")

    summary_post = pd.DataFrame({
        "Year": ["Total"],
        "Age": [""],
        "Portion of Year (%)": [""],
        "Wage Base": [""],
        "Gross Earnings": [""],
        "Adjusted Earnings": [f"${post_injury_adj_total:,.2f}"],
        "Present Value": [f"${post_injury_pv_total:,.2f}"]
    })
    summary_post.to_excel(writer, index=False, header=False, sheet_name="Post-Injury",
                          startrow=len(post_injury_df) + 2)

print(f"\nExport successful! The file has been saved as '{file_path}' in your working directory.")
print(f"Estimated retirement date (based on WLE): {estimated_retirement_dt.strftime('%Y-%m-%d')}")


Welcome to the Post-Trial Earnings Calculator (With Residual Offset)!

Export successful! The file has been saved as '/Users/chrisskerritt/EconomicWorkbook/John_Henry_Earnings_Tables.xlsx' in your working directory.
Estimated retirement date (based on WLE): 2073-01-24


In [6]:
#Healthcare Calculator

import pandas as pd
import os

def compute_healthcare_costs(
    report_date,
    wle_years,
    base_healthcare_cost,
    growth_rate,
    discount_rate
):
    """
    Computes a table of yearly (and partial-year) health care costs from 'report_date'
    through 'report_date + wle_years' using:
      - user-defined annual growth_rate on the health care cost
      - user-defined discount_rate for present value calculations
      - partial-year calculations for the first/last year

    Returns:
        df: A pandas DataFrame with columns:
            Year,
            Portion of Year (%),
            Health Insurance @ <growth_rate_in_%>,
            Yearly Value [(2) x (3)],
            Present Value @ <discount_rate_in_%>
    """

    # Convert to Timestamp
    start_dt = pd.to_datetime(report_date)
    # Calculate the retirement date from WLE in years
    days_for_wle = round(wle_years * 365.25)
    end_dt = start_dt + pd.Timedelta(days=days_for_wle)

    # Dynamically name the columns based on user inputs
    column_insurance = f"Health Insurance @ {growth_rate * 100:.1f}%"
    column_present_value = f"Present Value @ {discount_rate * 100:.1f}%"

    # Prepare data dictionary
    data = {
        "Year": [],
        "Portion of Year (%)": [],
        column_insurance: [],
        "Yearly Value [(2) x (3)]": [],
        column_present_value: []
    }

    current_year = start_dt.year
    current_annual_cost = base_healthcare_cost  # grows by (growth_rate) each year
    reference_start = start_dt  # discount from the date of report

    while True:
        # Identify the start and end of this calendar year
        year_start = pd.Timestamp(year=current_year, month=1, day=1)
        year_end = pd.Timestamp(year=current_year, month=12, day=31)

        # For the first iteration, ensure we don't start before 'start_dt'
        if year_start < start_dt:
            year_start = start_dt

        # For the final iteration, don't go beyond 'end_dt'
        if year_end > end_dt:
            year_end = end_dt

        # If this window is invalid (start beyond end), break
        if year_start > year_end:
            break

        # Calculate portion of the year
        days_in_period = (year_end - year_start).days + 1  # inclusive
        portion_of_year = days_in_period / 365.0

        # The (full) annual health insurance cost for this year
        annual_health_cost = current_annual_cost

        # The cost actually incurred in this partial year
        yearly_value = annual_health_cost * portion_of_year

        # Discounting back to the 'report_date'
        years_from_report = (year_start - reference_start).days / 365.0
        present_value = yearly_value / ((1 + discount_rate) ** years_from_report)

        # Append row
        data["Year"].append(current_year)
        data["Portion of Year (%)"].append(f"{portion_of_year * 100:.2f}%")
        data[column_insurance].append(f"${annual_health_cost:,.2f}")
        data["Yearly Value [(2) x (3)]"].append(f"${yearly_value:,.2f}")
        data[column_present_value].append(f"${present_value:,.2f}")

        # Grow health care cost by 'growth_rate' for the next calendar year
        current_annual_cost *= (1 + growth_rate)

        # Move to next year
        current_year += 1

        # If we've reached the final portion, break
        if year_end == end_dt:
            break

    # Build the final DataFrame
    df = pd.DataFrame(data)
    return df


def main():
    print("Welcome to the Health Care Cost Calculator!")

    # 1. Gather user inputs
    date_of_report = input("Enter the date of report (YYYY-MM-DD): ")
    wle_years = float(input("Enter the Work Life Expectancy (WLE) in years (e.g., 20): "))
    base_healthcare_cost = float(input("Enter the base annual health care cost (e.g., 5000): "))
    growth_rate_input = float(input("Enter the health care cost growth rate (as a percentage, e.g., 7): ")) / 100
    discount_rate_input = float(input("Enter the discount rate (as a percentage, e.g., 5): ")) / 100

    # 2. Compute the costs table
    df = compute_healthcare_costs(
        report_date=date_of_report,
        wle_years=wle_years,
        base_healthcare_cost=base_healthcare_cost,
        growth_rate=growth_rate_input,
        discount_rate=discount_rate_input
    )

    # 3. Calculate Totals
    def parse_dollar(value_str):
        return float(value_str.replace("$", "").replace(",", ""))

    # Dynamically retrieve column names for summation
    column_insurance = df.columns[2]  # "Health Insurance @ X%"
    column_pv = df.columns[-1]        # "Present Value @ X%"

    total_yearly_value = sum(parse_dollar(x) for x in df["Yearly Value [(2) x (3)]"])
    total_present_value = sum(parse_dollar(x) for x in df[column_pv])

    # 4. Export to Excel
    file_name = "HealthCare_Cost_Report.xlsx"
    file_path = os.path.join(os.getcwd(), file_name)

    with pd.ExcelWriter(file_path) as writer:
        df.to_excel(writer, index=False, sheet_name="Health Care Costs")

        # Write total row below
        summary_df = pd.DataFrame({
            "Year": ["Total"],
            "Portion of Year (%)": [""],
            column_insurance: [""],
            "Yearly Value [(2) x (3)]": [f"${total_yearly_value:,.2f}"],
            column_pv: [f"${total_present_value:,.2f}"]
        })
        summary_df.to_excel(
            writer,
            index=False,
            header=False,
            sheet_name="Health Care Costs",
            startrow=len(df) + 2
        )

    print(f"\nCalculation complete! Your file has been saved to: {file_path}")


# Run the script if executed directly
if __name__ == "__main__":
    main()


Welcome to the Health Care Cost Calculator!

Calculation complete! Your file has been saved to: /Users/chrisskerritt/EconomicWorkbook/HealthCare_Cost_Report.xlsx


In [1]:
# Pension Benefits Year-over-Year Analysis

#!/usr/bin/env python
# coding: utf-8

"""
Jupyter Notebook Script for Calculating and Reporting
Lost Earnings & Non-Wage Compensation (Union Construction Worker)

Author: [Your Name]
Date: [Date]

This script uses ipywidgets to collect user input and generate
a summarized economic loss analysis based on the template inspired
by Tinari and Betz (2013). The final output can be displayed
as a formatted report within the notebook.
"""

import ipywidgets as widgets
from IPython.display import display, Markdown, HTML
import pandas as pd
import math

# ------------------------------------------------------------------------------------
# 1. Introduction
# ------------------------------------------------------------------------------------
intro_text = """
## Lost Earnings & Non-Wage Compensation Calculator

This Jupyter Notebook collects the user’s inputs regarding a union
construction worker's wage history, hours worked, and fringe benefit
contribution rates. It then calculates the estimated losses in wages,
annuity contributions, vacation benefits, welfare/health coverage,
and pension benefits, following the considerations outlined by
Tinari and Betz (2013).

**Instructions:**
1. Enter the general case information (Union Name, CBA Dates, etc.).
2. Use the table input to add or remove year-by-year wage/hour data.
3. Input the relevant fringe benefit rates (annuity, vacation, etc.).
4. Generate the final report to see a summarized calculation of losses.

**Note:** This script is intended for demonstration and educational purposes.
Further customization may be needed to address unique case details and
additional benefits or offsets.
"""
display(Markdown(intro_text))

# ------------------------------------------------------------------------------------
# 2. General Information Widgets
# ------------------------------------------------------------------------------------
union_name = widgets.Text(
    value='Local XXX of the [Union Name]',
    description='Union:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

cba_effective_dates = widgets.Text(
    value='July 1, 20XX – June 30, 20YY',
    description='CBA Dates:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

display(Markdown("### General Case Information"))
display(union_name, cba_effective_dates)

# ------------------------------------------------------------------------------------
# 3. Data Input for Yearly Wages and Hours
# ------------------------------------------------------------------------------------
# We’ll store the data in a pandas DataFrame for convenience.
df_columns = ['Year', 'Hourly Wage Rate', 'Hours Worked', 'Gross Earnings (W-2)']

# Initialize an empty DataFrame
data_df = pd.DataFrame(columns=df_columns)

# Define widgets for row entry
year_input = widgets.IntText(
    value=2020,
    description='Year:',
    style={'description_width': '50px'},
    layout=widgets.Layout(width='180px')
)

wage_rate_input = widgets.FloatText(
    value=40.00,
    description='Hourly Wage:',
    style={'description_width': '90px'},
    layout=widgets.Layout(width='200px')
)

hours_worked_input = widgets.IntText(
    value=1800,
    description='Hours:',
    style={'description_width': '50px'},
    layout=widgets.Layout(width='160px')
)

earnings_input = widgets.FloatText(
    value=72000.00,
    description='Gross W-2:',
    style={'description_width': '80px'},
    layout=widgets.Layout(width='220px')
)

# Button to add row
add_row_button = widgets.Button(
    description='Add/Update Row',
    button_style='success',
    icon='plus'
)

# Button to remove row
remove_row_button = widgets.Button(
    description='Remove Year',
    button_style='warning',
    icon='minus'
)

# Table display widget (we’ll update this after changes)
output_table = widgets.Output()

def refresh_table_display():
    with output_table:
        output_table.clear_output()
        if not data_df.empty:
            display(data_df.style.hide_index())
        else:
            display(Markdown("*No year data currently in the table.*"))

def add_or_update_row(_):
    """
    Adds or updates a row in the DataFrame based on the year.
    """
    global data_df
    y = year_input.value

    # Check if the year already exists
    existing_index = data_df.index[data_df['Year'] == y].tolist()
    if existing_index:
        # Update that row
        idx = existing_index[0]
        data_df.loc[idx, 'Hourly Wage Rate'] = wage_rate_input.value
        data_df.loc[idx, 'Hours Worked'] = hours_worked_input.value
        data_df.loc[idx, 'Gross Earnings (W-2)'] = earnings_input.value
    else:
        # Append a new row
        new_row = {
            'Year': y,
            'Hourly Wage Rate': wage_rate_input.value,
            'Hours Worked': hours_worked_input.value,
            'Gross Earnings (W-2)': earnings_input.value
        }
        data_df = data_df.append(new_row, ignore_index=True)

    data_df.sort_values(by='Year', inplace=True)
    data_df.reset_index(drop=True, inplace=True)
    refresh_table_display()

def remove_row(_):
    """
    Removes the row with the matching year from the DataFrame.
    """
    global data_df
    y = year_input.value
    data_df = data_df[data_df['Year'] != y]
    data_df.reset_index(drop=True, inplace=True)
    refresh_table_display()

add_row_button.on_click(add_or_update_row)
remove_row_button.on_click(remove_row)

display(Markdown("### Year-by-Year Data Entry"))
display(widgets.HBox([year_input, wage_rate_input, hours_worked_input, earnings_input]))
display(widgets.HBox([add_row_button, remove_row_button]))
display(output_table)
refresh_table_display()

# ------------------------------------------------------------------------------------
# 4. Fringe Benefit Inputs
# ------------------------------------------------------------------------------------
fringe_instructions = """
### Fringe Benefit & Pension Inputs

Please enter the relevant employer contribution rates and assumptions
about each fringe benefit. If the union CBA indicates that contributions
are made per hour (straight time only), enter the rates below. If the
Vacation Fund is already included in the W-2 wages, set that rate to 0.

**Welfare/Health**: If you have an annual or monthly replacement cost
(COBRA or similar), enter that figure. Otherwise, you can input an
hourly-based approximation.

**Pension Formula**: For simplicity, this example collects a single
hourly *pension contribution rate*. If the actual plan is more
complex (e.g., 1% of total contributions per hour for monthly benefits),
we can approximate or adapt the code as needed.
"""
display(Markdown(fringe_instructions))

# Annuity
annuity_label = widgets.Label("Annuity Fund Rate (per hour):")
annuity_rate = widgets.FloatText(
    value=6.60,
    description='',
    layout=widgets.Layout(width='150px')
)

# Vacation
vacation_label = widgets.Label("Vacation Fund Rate (per hour):")
vacation_rate = widgets.FloatText(
    value=7.10,
    description='',
    layout=widgets.Layout(width='150px')
)

# Welfare
welfare_label = widgets.Label("Welfare/Health Rate (per hour or monthly COBRA):")
welfare_rate = widgets.FloatText(
    value=10.25,
    description='',
    layout=widgets.Layout(width='150px')
)

# Pension
pension_label = widgets.Label("Pension Fund Rate (per hour):")
pension_rate = widgets.FloatText(
    value=11.81,
    description='',
    layout=widgets.Layout(width='150px')
)

fringe_box = widgets.VBox([
    widgets.HBox([annuity_label, annuity_rate]),
    widgets.HBox([vacation_label, vacation_rate]),
    widgets.HBox([welfare_label, welfare_rate]),
    widgets.HBox([pension_label, pension_rate]),
])

display(fringe_box)

# ------------------------------------------------------------------------------------
# 5. Calculation & Report Generation
# ------------------------------------------------------------------------------------

calc_button = widgets.Button(
    description='Generate Report',
    button_style='primary',
    icon='file'
)

report_output = widgets.Output()

def generate_calculations():
    """
    Perform the year-over-year calculations for wages and fringe benefits.
    Returns a DataFrame summarizing the results.
    """
    global data_df

    if data_df.empty:
        return pd.DataFrame()

    # Copy for calculations
    calc_df = data_df.copy()

    # For illustration: assume all contributions are per *straight-time* hour
    # The user can adapt if some or all contributions apply to overtime rates.
    calc_df['Annuity Contribution'] = calc_df['Hours Worked'] * annuity_rate.value
    calc_df['Vacation Contribution'] = calc_df['Hours Worked'] * vacation_rate.value

    # Welfare coverage is trickier; we assume an hourly-based approach here
    # If the user enters a monthly rate, they'd need to adapt how it's multiplied.
    calc_df['Welfare Contribution'] = calc_df['Hours Worked'] * welfare_rate.value

    # Pension: similarly, multiply hours by the pension rate
    calc_df['Pension Contribution'] = calc_df['Hours Worked'] * pension_rate.value

    return calc_df

def create_report_html(calc_df):
    """
    Generates an HTML-formatted report summarizing the data, calculations,
    and references. Combines paragraph text with embedded tables.
    """
    if calc_df.empty:
        return "<h3>No data available for calculation.</h3>"

    # Summaries
    total_base_wages = calc_df['Gross Earnings (W-2)'].sum()
    total_annuities = calc_df['Annuity Contribution'].sum()
    total_vacation = calc_df['Vacation Contribution'].sum()
    total_welfare  = calc_df['Welfare Contribution'].sum()
    total_pension  = calc_df['Pension Contribution'].sum()

    # Build a year-by-year table of fringe results
    fringe_table = calc_df[[
        'Year',
        'Hours Worked',
        'Gross Earnings (W-2)',
        'Annuity Contribution',
        'Vacation Contribution',
        'Welfare Contribution',
        'Pension Contribution'
    ]].round(2)

    # Convert to HTML (with an inline style for better readability in notebooks)
    fringe_table_html = fringe_table.to_html(index=False, justify='center')

    # Construct HTML for the final report
    report_html = f"""
    <h2>Economic Loss Analysis: Lost Earnings & Fringe Benefits</h2>
    <p><strong>Union Name:</strong> {union_name.value}<br>
    <strong>CBA Effective Dates:</strong> {cba_effective_dates.value}</p>

    <h3>1. Introduction and Purpose</h3>
    <p>This report evaluates the economic damages for a union construction worker
    following the methodology of Tinari and Betz (2013). The analysis incorporates
    the Claimant’s base wages and typical non-wage compensation (annuity, vacation,
    welfare, and pension).</p>

    <h3>2. Data & Methodology</h3>
    <ul>
      <li>Historic wage data (W-2) and hours worked were used to compute actual earnings.</li>
      <li>Fringe benefit contribution rates were applied to straight-time hours to estimate
          the annual value of each non-wage component.</li>
      <li>The Welfare Fund’s lost value is approximated using a simple hourly approach.
          Adjustments may be necessary if a monthly COBRA premium is more accurate.</li>
      <li>The Pension Fund is calculated based on a per-hour contribution. If an actual
          defined benefit formula exists, further refinement is recommended to capture
          monthly retirement benefits (see Tinari and Betz, 2013).</li>
    </ul>

    <h3>3. Year-Over-Year Calculations</h3>
    {fringe_table_html}

    <h3>4. Summary of Totals</h3>
    <ul>
      <li><strong>Total Base Wages (Gross Earnings):</strong> ${total_base_wages:,.2f}</li>
      <li><strong>Total Annuity Contributions:</strong> ${total_annuities:,.2f}</li>
      <li><strong>Total Vacation Contributions:</strong> ${total_vacation:,.2f}</li>
      <li><strong>Total Welfare Contributions:</strong> ${total_welfare:,.2f}</li>
      <li><strong>Total Pension Contributions:</strong> ${total_pension:,.2f}</li>
    </ul>

    <h3>5. References</h3>
    <ul>
      <li>Martin, G. D. (2011). <em>Determining Economic Damages</em>. Costa Mesa, CA: James Publishing.</li>
      <li>Rodgers, J. D. (2002). “Valuing Losses of Pension Benefits.” <em>Journal of Forensic Economics</em>, 15(2), 205–232.</li>
      <li>Rodgers, J. D. (2007). “Fringe Benefit Losses.” In Brookshire, M. L., Slesnick, F., & Ward, J. O. (Eds.),
      <em>The Plaintiff and Defense Attorney’s Guide to Understanding Economic Damages</em>, Chapter 4. Tucson, AZ: Lawyers & Judges Publishing.</li>
      <li>Tinari, F., & Betz, K. (2013). “Valuing Non-Wage Compensation of Private Sector Labor Union Workers
      in the Construction Trades.” <em>Journal of Forensic Economics</em>, 24(2), 205–220.</li>
    </ul>
    """
    return report_html

def on_generate_report_clicked(_):
    """
    When the Generate Report button is clicked, calculate all fields,
    then display an HTML-based final summary report in the notebook.
    """
    calc_df = generate_calculations()
    html_report = create_report_html(calc_df)
    with report_output:
        report_output.clear_output()
        display(HTML(html_report))

calc_button.on_click(on_generate_report_clicked)

display(Markdown("### Generate Final Report"))
display(calc_button)
display(report_output)




Welcome to the Pension Benefits Calculator (Based on Tinari & Betz, 2013)

Calculation complete! Report saved as: /Users/chrisskerritt/EconomicWorkbook/Pension_Benefits_Report.xlsx
